# Project Setup and Data Processing

This notebook handles the initial setup of the project environment and data processing pipeline. Follow these steps in order.

## 1. Environment Setup

First, ensure you have installed all required dependencies:

In [1]:
!pip install -r requirements.txt

DEPRECATION: Loading egg at /Users/Angusf777/anaconda3/lib/python3.11/site-packages/modelindex-0.0.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/Angusf777/anaconda3/lib/python3.11/site-packages/ordered_set-4.1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/Angusf777/anaconda3/lib/python3.11/site-packages/model_index-0.1.11-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/Angusf777/anaconda3/lib/python3.11/site-packages/e

## 2. Create Directory Structure

In [2]:
import os
from pathlib import Path

# Create necessary directories
directories = ['Data', 'Tools', 'Checkpoints']
base_path = Path.cwd()

for dir_name in directories:
    dir_path = base_path / dir_name
    if not dir_path.exists():
        dir_path.mkdir(parents=True)
        print(f"Created {dir_name} directory")
    else:
        print(f"{dir_name} directory already exists")

Created Data directory
Tools directory already exists
Checkpoints directory already exists


## 3. Download and Extract Kickstarter Dataset

Download the Kickstarter dataset from webrobots.io and extract it to the Data directory.

In [3]:
import requests
import gzip
from tqdm import tqdm
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Dataset URL (@https://webrobots.io/kickstarter-datasets/)
dataset_url = 'https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2024-12-12T03_20_04_455Z.json.gz'
output_file = Path('Data/Kickstarter_2024-12-12T03_20_04_455Z.json')

def download_and_extract():
    """Download and extract the Kickstarter dataset."""
    if output_file.exists():
        logger.info(f"File already exists at {output_file}")
        return
    
    logger.info("Downloading dataset...")
    response = requests.get(dataset_url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    # Download compressed file
    compressed_file = output_file.with_suffix('.json.gz')
    with open(compressed_file, 'wb') as f:
        with tqdm(total=total_size, unit='iB', unit_scale=True) as pbar:
            for data in response.iter_content(chunk_size=8192):
                size = f.write(data)
                pbar.update(size)
    
    logger.info("Extracting gzipped file...")
    with gzip.open(compressed_file, 'rb') as f_in:
        with open(output_file, 'wb') as f_out:
            f_out.write(f_in.read())
    
    # Clean up compressed file
    compressed_file.unlink()
    logger.info("Download and extraction complete!")

download_and_extract()

2025-01-15 22:06:44,517 - INFO - Downloading dataset...
100%|██████████| 269M/269M [00:42<00:00, 6.36MiB/s]   
2025-01-15 22:07:27,797 - INFO - Extracting gzipped file...
2025-01-15 22:07:35,515 - INFO - Download and extraction complete!


## 4. Process Data with check_duplicates.py, filter_kickstarter.py and make_WebDatabase.py

Run both scripts to process the raw dataset and create the web database.

In [4]:
import subprocess
import sys
from pathlib import Path

def run_script_with_args(script_path: Path, input_path: Path, output_path: Path, stats_path: Path = None):
    """Run a Python script with command line arguments."""
    try:
        logger.info(f"Running {script_path.name}...")
        
        # Build command with required arguments
        cmd = [
            sys.executable,
            str(script_path),
            '--input', str(input_path),
            '--output', str(output_path)
        ]
        
        # Add stats path if provided
        if stats_path:
            cmd.extend(['--stats', str(stats_path)])
        
        # Run the command
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        # Print output
        if result.stdout:
            print(result.stdout)
        if result.stderr:
            print(result.stderr)
            
        # Check if output file was created
        if output_path.exists():
            size_mb = output_path.stat().st_size / (1024 * 1024)
            logger.info(f"Successfully created {output_path.name} ({size_mb:.2f} MB)")
        else:
            raise FileNotFoundError(f"Expected output file {output_path} was not created")
            
    except Exception as e:
        logger.error(f"Error running {script_path.name}: {e}")
        raise

# Step 1: Remove duplicates
logger.info("Step 1: Removing duplicates...")
duplicate_script = Path('Tools/check_duplicates.py')
duplicate_input = Path('Data/Kickstarter_2024-12-12T03_20_04_455Z.json')
duplicate_output = Path('Data/Kickstarter_removed_duplicates.json')
duplicate_stats = Path('Data/duplicate_stats.json')

if duplicate_script.exists():
    run_script_with_args(duplicate_script, duplicate_input, duplicate_output, duplicate_stats)
else:
    logger.error(f"Script not found: {duplicate_script}")

# Step 2: Run filter on deduplicated data
logger.info("Step 2: Filtering deduplicated data...")
filter_script = Path('Tools/filter_kickstarter.py')
filter_input = Path('Data/Kickstarter_removed_duplicates.json')
filter_output = Path('Data/Kickstarter_filtered.json')
filter_stats = Path('Data/filtering_stats.json')

if filter_script.exists():
    run_script_with_args(filter_script, filter_input, filter_output, filter_stats)
else:
    logger.error(f"Script not found: {filter_script}")

# Step 3: Create web database from deduplicated data
logger.info("Step 3: Creating web database from deduplicated data...")
web_db_script = Path('Tools/make_WebDatabase.py')
web_db_input = Path('Data/Kickstarter_removed_duplicates.json')
web_db_output = Path('Data/website_database.json')
web_db_stats = Path('Data/web_processing_stats.json')

if web_db_script.exists():
    run_script_with_args(web_db_script, web_db_input, web_db_output, web_db_stats)
else:
    logger.error(f"Script not found: {web_db_script}")

# Verify all files were created
required_files = [
    duplicate_output,
    duplicate_stats,
    filter_output,
    filter_stats,
    web_db_output,
    web_db_stats
]

logger.info("\nVerifying created files:")
for file in required_files:
    if file.exists():
        size_mb = file.stat().st_size / (1024 * 1024)
        logger.info(f"✓ {file.name} exists ({size_mb:.2f} MB)")
    else:
        logger.error(f"✗ {file.name} is missing")

2025-01-15 22:07:35,574 - INFO - Step 1: Removing duplicates...
2025-01-15 22:07:35,577 - INFO - Running check_duplicates.py...


2025-01-15 22:08:17,365 - INFO - Successfully created Kickstarter_removed_duplicates.json (1326.20 MB)
2025-01-15 22:08:17,367 - INFO - Step 2: Filtering deduplicated data...
2025-01-15 22:08:17,368 - INFO - Running filter_kickstarter.py...



Duplicate Removal Summary:
-------------------------
Total projects processed: 231,290
Duplicates removed: 29,380
Unique projects remaining: 201,910

Deduplicated data saved to: Data/Kickstarter_removed_duplicates.json
Statistics saved to: Data/duplicate_stats.json

2025-01-15 22:07:35,774 - INFO - Starting duplicate removal process...
2025-01-15 22:08:12,045 - INFO - Duplicate removal completed!



2025-01-15 22:08:52,928 - INFO - Successfully created Kickstarter_filtered.json (1250.65 MB)
2025-01-15 22:08:52,929 - INFO - Step 3: Creating web database from deduplicated data...
2025-01-15 22:08:52,929 - INFO - Running make_WebDatabase.py...


2025-01-15 22:08:52,920 - INFO - 
Filtering completed!
2025-01-15 22:08:52,920 - INFO - Total processed: 201910
2025-01-15 22:08:52,920 - INFO - Included: 189796
2025-01-15 22:08:52,920 - INFO - Excluded: 12114
2025-01-15 22:08:52,920 - INFO - 
Canceled projects summary:
2025-01-15 22:08:52,920 - INFO - Total canceled: 8937
2025-01-15 22:08:52,920 - INFO - Converted to failed: 5738
2025-01-15 22:08:52,920 - INFO - Excluded (>60% remaining): 3199
2025-01-15 22:08:52,920 - INFO - Invalid timestamps: 0
2025-01-15 22:08:52,920 - INFO - 
Detailed statistics saved to: Data/filtering_stats.json



2025-01-15 22:09:08,376 - INFO - Successfully created website_database.json (166.01 MB)
2025-01-15 22:09:08,377 - INFO - 
Verifying created files:
2025-01-15 22:09:08,378 - INFO - ✓ Kickstarter_removed_duplicates.json exists (1326.20 MB)
2025-01-15 22:09:08,378 - INFO - ✓ duplicate_stats.json exists (0.00 MB)
2025-01-15 22:09:08,380 - INFO - ✓ Kickstarter_filtered.json exists (1250.65 MB)
2025-01-15 22:09:08,380 - INFO - ✓ filtering_stats.json exists (0.02 MB)
2025-01-15 22:09:08,380 - INFO - ✓ website_database.json exists (166.01 MB)
2025-01-15 22:09:08,381 - INFO - ✓ web_processing_stats.json exists (0.00 MB)


2025-01-15 22:08:53,192 - INFO - Starting to process Kickstarter data...
2025-01-15 22:08:53,244 - INFO - Processed 1000 records successfully...
2025-01-15 22:08:53,294 - INFO - Processed 2000 records successfully...
2025-01-15 22:08:53,348 - INFO - Processed 3000 records successfully...
2025-01-15 22:08:53,399 - INFO - Processed 4000 records successfully...
2025-01-15 22:08:53,444 - INFO - Processed 5000 records successfully...
2025-01-15 22:08:53,492 - INFO - Processed 6000 records successfully...
2025-01-15 22:08:53,539 - INFO - Processed 7000 records successfully...
2025-01-15 22:08:53,589 - INFO - Processed 8000 records successfully...
2025-01-15 22:08:53,635 - INFO - Processed 9000 records successfully...
2025-01-15 22:08:53,682 - INFO - Processed 10000 records successfully...
2025-01-15 22:08:53,728 - INFO - Processed 11000 records successfully...
2025-01-15 22:08:53,773 - INFO - Processed 12000 records successfully...
2025-01-15 22:08:53,820 - INFO - Processed 13000 records suc

## 5. Verify Setup

Check that all required files have been created successfully.

In [5]:
required_files = [
    'Kickstarter_2024-12-12T03_20_04_455Z.json',  # Original dataset
    'Kickstarter_removed_duplicates.json',         # Deduplicated dataset
    'duplicate_stats.json',                        # Duplicate removal statistics
    'Kickstarter_filtered.json',                   # Filtered dataset
    'filtering_stats.json',                        # Filtering statistics
    'website_database.json',                       # Web database
    'web_processing_stats.json'                    # Web processing statistics
]

logger.info("Checking for required files:")
for file in required_files:
    file_path = Path('Data') / file
    if file_path.exists():
        size_mb = file_path.stat().st_size / (1024 * 1024)
        logger.info(f"✓ {file} exists ({size_mb:.2f} MB)")
    else:
        logger.error(f"✗ {file} is missing")

2025-01-15 22:09:08,391 - INFO - Checking for required files:
2025-01-15 22:09:08,392 - INFO - ✓ Kickstarter_2024-12-12T03_20_04_455Z.json exists (1465.22 MB)
2025-01-15 22:09:08,393 - INFO - ✓ Kickstarter_removed_duplicates.json exists (1326.20 MB)
2025-01-15 22:09:08,393 - INFO - ✓ duplicate_stats.json exists (0.00 MB)
2025-01-15 22:09:08,394 - INFO - ✓ Kickstarter_filtered.json exists (1250.65 MB)
2025-01-15 22:09:08,394 - INFO - ✓ filtering_stats.json exists (0.02 MB)
2025-01-15 22:09:08,395 - INFO - ✓ website_database.json exists (166.01 MB)
2025-01-15 22:09:08,395 - INFO - ✓ web_processing_stats.json exists (0.00 MB)


In [ ]:
import subprocess

# Run create_pre_input.py
subprocess.run(["python", "create_pre_input.py"], check=True)